* hazelcast_distributed-map.py

In [ ]:
import hazelcast
import time


def distribute_1000key():
    for key in range(1000): dist_map.set(key, key)


# Функція для інкременту значення лічильника без Lock, з простим читанням і записом
def without_lock(key):
    dist_map.put(key, 0)
    for _ in range(10000):
        counter = dist_map.get(key) + 1
        dist_map.put(key, counter)


# Функція для інкременту значення лічильника з використанням Lock-мапи (map.lock)
def pessimistic_map(key):
    if (not dist_map.contains_key(key)): dist_map.put(key, 0)
    for _ in range(10000):
        dist_map.lock(key)
        try:
            counter = dist_map.get(key) + 1
            dist_map.put(key, counter)
        finally:
            dist_map.unlock(key)


# Функція для інкременту значення з використанням механізму заміни значень (replace_if_same)
def optimistic_replace(key):
    if (not dist_map.contains_key(key)): dist_map.put(key, 0)
    for _ in range(10000):
        while True:
            oldcounter = dist_map.get(key)
            newcounter = oldcounter + 1
            if dist_map.replace_if_same(key, oldcounter, newcounter): break


if __name__ == "__main__":
    # Підключення до Hazelcast через Python-клієнта
    hz = hazelcast.HazelcastClient(cluster_name="distributed-map-cluster",
                                   client_name="hazelcast-connection")
    dist_map = hz.get_map("lab-distributed-map").blocking()
    print("Successful connection to Hazelcast!")

    # Функція для запису (фактичного розподілу) 
    # 1000 значень з ключами від 0 до 1000
    # distribute_1000key()  # Закоментовано
    
    # Використання ключа для розподіленого лічильника
    main_counter = 0
    print("Initial counter value:", main_counter)

    # Завдання до виконання
    tasks = [[without_lock, "a. No Lock for Increment"],
             [pessimistic_map, "b. Map Lock for Increment"],
             [optimistic_replace, "c. Replace if Same Increment"]]

    for task, name in tasks:
        print(f"\n{'-' * 50}\nExecuting {name}...\n{'-' * 50}")
        
        start = time.time()
        task(task.__name__)
            
        print(f"Task '{name}' executed in: {time.time() - start:.2f} seconds")
        print(f"Final counter value: {dist_map.get(task.__name__)}")
        print(f"{'-' * 50}\n")

    # Завершення роботи клієнта
    hz.shutdown()

---

* writer-bounded_queue.py

In [ ]:
import hazelcast

hz = hazelcast.HazelcastClient(cluster_name="distributed-map-cluster")
bounded_queue = hz.get_queue("lab-distributed-queue").blocking()
print("Successful connection to Hazelcast!")

for item in range(1, 101):
    bounded_queue.put(item)
    print(f"| Item: {item}\t | Size: {bounded_queue.size()}\t |\n"
          f"| Remaining Capacity: [{bounded_queue.remaining_capacity()}]\t |\n")

bounded_queue.put(-1)
hz.shutdown()

* reader-bounded_queue.py

In [ ]:
import hazelcast

hz = hazelcast.HazelcastClient(cluster_name="distributed-map-cluster")
bounded_queue = hz.get_queue("lab-distributed-queue").blocking()
print("Successful connection to Hazelcast!")

item = 0

while item != -1:
    item = bounded_queue.take()
    print(f"> Taken from queue: {item} \n")

else:
    bounded_queue.put(-1)
    hz.shutdown()

---